In [1]:
#@title Step 2: Cloning the OpenAI GPT-2 Repository 
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (236/236), 4.38 MiB | 652.00 KiB/s, done.
Resolving deltas: 100% (124/124), done.


In [3]:
#@title Step 3: Installing the requirements
import os                     # when the VM restarts import os necessary
os.chdir("gpt-2")    
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 395.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
^C
ERROR: Operation cancelled by user


In [3]:
#@title Step 4 Checking the Version of TensorFlow 
#Colab has tf 1.x and tf 2.x installed
#Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [4]:
#@title Step 5: Downloading the 345M parameter GPT-2 Model
# run code and send argument
import os # after runtime is restarted
os.chdir("/content/gpt-2")
!python3 download_model.py '345M' 

Fetching checkpoint: 1.00kit [00:00, 945kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 3.97Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 944kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:27, 51.2Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 9.49Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 3.13Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.48Mit/s]                                                       


In [5]:
#@title Step 6: Printing UTF encoded text to the console
!export PYTHONIOENCODING=UTF-8

In [6]:
#@title Step 7: Project Source Code
import os # import after runtime is restarted
os.chdir("/content/gpt-2/src")

In [7]:
#@title Step 7a: Interactive Conditional Samples (src)
#Project Source Code for Interactive Conditional Samples:
# /content/gpt-2/src/interactive_conditional_samples.py file 
import json
import os
import numpy as np
import tensorflow as tf

In [8]:
#@title Step 7b: Importing model sample encoder
import model, sample, encoder
#if following message:
#ModuleNotFoundError: No module named 'tensorflow.contrib'
#then go back and run Step 2 Checking TensorFlow version 

In [9]:
#@title Step 8: Defining the model
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [10]:
#@title Step 9: Interacting with GPT-2 
interact_model('345M',None,1,1,300,1,0,'/content/gpt-2/models')





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
======================================== SAMPLE 1 ========================================
 But to hold to sense alone, as to the only thing capable of constituting our perfection, is the very aim wherein nature herself establishes herself. This shall never be the final end of human reason, as I apprehend this to be; unless, indeed, it begins from spirit, and and passes through man to no other end: therefore intellectual ideas don't contemplate any hell, the existence of which the Saccadic demon of Illustration would require for perfection.

Now, if you should see it thus, it will seem rather to refute the sensible traits of Plato who posited nature as an objective object, when she was anathema to h

KeyboardInterrupt: ignored